# Detecting Climate Disinformation based on text from news articles : Explore the Data 

## Import Python Packages

In [31]:
from datasets import (load_dataset_builder,
                      load_dataset,
                      get_dataset_split_names)

from pprint import pprint

print("All packages imported successfully!")

All packages imported successfully!


## Explore the Dataset

In [4]:
ds_builder = load_dataset_builder("QuotaClimat/frugalaichallenge-text-train")

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

In [7]:
# Inspect dataset features

ds_builder.info.features

{'quote': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None),
 'source': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'subsource': Value(dtype='string', id=None),
 'id': Value(dtype='null', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [9]:
# Inspect the number of splits available for the dataset

get_dataset_split_names("QuotaClimat/frugalaichallenge-text-train")

['train']

In [10]:
# Download the dataset 

dataset = load_dataset("QuotaClimat/frugalaichallenge-text-train", split="train")

train.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6091 [00:00<?, ? examples/s]

In [28]:
print(f"ROWS: {dataset.num_rows}")
print(f"COLUMNS: {dataset.num_columns}")

ROWS: 6091
COLUMNS: 8


In [29]:
print(f"COLUMN NAMES: {dataset.column_names}")

COLUMN NAMES: ['quote', 'label', 'source', 'url', 'language', 'subsource', 'id', '__index_level_0__']


In [35]:
# EXAMPLE ROW
pprint(dataset[0])

{'__index_level_0__': 0,
 'id': None,
 'label': '5_science_unreliable',
 'language': 'en',
 'quote': 'There is clear, compelling evidence that many of the major '
          'conclusions of the IPCC, your new religions constantly-changing '
          'Holy Book, are based on evidence that has been fabricated. The '
          'hockey stick graph that purported to abolish the mediaeval warm '
          'period is just one example.',
 'source': 'FLICC',
 'subsource': 'CARDS',
 'url': 'https://huggingface.co/datasets/fzanartu/FLICCdataset'}


In [37]:
# Get unique values from the 'language' column
dataset.unique('language')

['en']

In [39]:
# Get the count of unique URLs
len(dataset.unique('url'))

780

In [40]:
# Get unique values from the 'source' column
dataset.unique('source')

['FLICC', 'Desmog']

In [41]:
# Get unique values from the 'subsource' column
dataset.unique('subsource')

['CARDS',
 'hamburg_test1',
 'hamburg_test3',
 'jindev',
 'jintrain',
 'hamburg_test2',
 'Alhindi_train',
 'jintest',
 'Alhindi_dev',
 'Alhindi_test',
 None]

In [42]:
# Get the count of unique quotes
len(dataset.unique('quote'))

6091

In [43]:
# Get unique values from the 'label' column
dataset.unique('label')

['5_science_unreliable',
 '1_not_happening',
 '4_solutions_harmful_unnecessary',
 '0_not_relevant',
 '6_proponents_biased',
 '7_fossil_fuels_needed',
 '2_not_human',
 '3_not_bad']

In [44]:
# Get unique values from the 'id' column
dataset.unique('id')

[None]

In [45]:
# Get the count of unique index values.
len(dataset.unique('__index_level_0__'))

6091